# **Model Training**

In [0]:
from os.path import join
from google.colab import drive
# Montar el DRIVE
ROOT = "/content/drive"
drive.mount(ROOT)# Apuntar a la carpeta de trabajo
PROJ = "My Drive/MSI-MSS_project" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.compat.v1.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.91.62.42:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8679310372501781210),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5423930887333250103),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, -1072185786516004602),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8239468930526599291),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, -2983584815579244607),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 7783963565623042973),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1910666631864057584),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 6264946886908400831),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, -62526157241088

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

"""
resnet_model = tf.contrib.tpu.keras_to_tpu_model(
    resnet_model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
"""

tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver) 

INFO:tensorflow:Initializing the TPU system: grpc://10.91.62.42:8470
INFO:tensorflow:Clearing out eager caches
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:te

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import backend as K

In [0]:
np.random.seed(1000)

#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(2)) #cambiado por mi
model.add(Activation('softmax'))


model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy']) 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 384)        

In [0]:
dataset = pd.read_pickle(join(PROJECT_PATH,'final_subset_Color.pkl'))

In [0]:
dataset.head()

,label,image_array
0,MSI,"[[[207, 131, 178], [216, 161, 201], [178, 126,..."
1,MSI,"[[[216, 136, 187], [205, 121, 173], [195, 108,..."
2,MSI,"[[[254, 246, 244], [253, 238, 243], [255, 229,..."
3,MSI,"[[[159, 101, 152], [186, 129, 182], [210, 155,..."
4,MSI,"[[[173, 105, 156], [203, 141, 182], [203, 159,..."


In [0]:
dataset = dataset.apply(pd.to_numeric, args=("ignore",))

In [0]:
dataset

,label,image_array
0,MSI,"[[[207, 131, 178], [216, 161, 201], [178, 126,..."
1,MSI,"[[[216, 136, 187], [205, 121, 173], [195, 108,..."
2,MSI,"[[[254, 246, 244], [253, 238, 243], [255, 229,..."
3,MSI,"[[[159, 101, 152], [186, 129, 182], [210, 155,..."
4,MSI,"[[[173, 105, 156], [203, 141, 182], [203, 159,..."
...,...,...
3995,MSS,"[[[236, 187, 217], [233, 175, 190], [206, 173,..."
3996,MSS,"[[[242, 236, 240], [242, 236, 240], [242, 236,..."
3997,MSS,"[[[154, 80, 129], [126, 57, 104], [106, 42, 92..."
3998,MSS,"[[[204, 122, 171], [202, 120, 168], [207, 128,..."


In [0]:
X = np.array(list(dataset["image_array"]))
#X = np.concatenate((cats,dogs), axis=2).swapaxes(2,0)
y = np.concatenate((np.zeros(2000),np.ones(2000)))
print("Shapes X={} y={}".format(X.shape,y.shape))

Shapes X=(4000, 224, 224, 3) y=(4000,)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [0]:
y_test.shape

(800,)

In [0]:
# Prepare data to feed the NN
num_classes = 2
img_rows, img_cols = 224, 224

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (3200, 224, 224, 3)
3200 train samples
800 test samples


In [0]:
# Fit the NN
batch_size = 20
epochs = 10

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 3200 samples, validate on 800 samples
Epoch 1/10
3200/3200 [==============================] - 521s 163ms/step - loss: 0.6984 - accuracy: 0.4953 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 2/10
3200/3200 [==============================] - 522s 163ms/step - loss: 0.6938 - accuracy: 0.4988 - val_loss: 0.6930 - val_accuracy: 0.5100
Epoch 3/10
3200/3200 [==============================] - 512s 160ms/step - loss: 0.6940 - accuracy: 0.4872 - val_loss: 0.6934 - val_accuracy: 0.4900
Epoch 4/10
3200/3200 [==============================] - 516s 161ms/step - loss: 0.6934 - accuracy: 0.4925 - val_loss: 0.6934 - val_accuracy: 0.4900
Epoch 5/10
3200/3200 [==============================] - 515s 161ms/step - loss: 0.6937 - accuracy: 0.4975 - val_loss: 0.6930 - val_accuracy: 0.5100
Epoch 6/10
3200/3200 [==============================] - 519s 162ms/step - loss: 0.6934 - accuracy: 0.5031 - val_loss: 0.6935 - val_accuracy: 0.4900
Epoch 7/10
3200/3200 [==============================] - 505s 158m

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6931954050064086
Test accuracy: 0.49000000953674316
